## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
from tqdm.notebook import tqdm

# Import API key
from config import g_key



In [2]:
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Geraldton,AU,-28.7667,114.6000,93.20,24,3,19.57,clear sky
1,1,Hasaki,JP,35.7333,140.8333,41.00,87,75,6.91,broken clouds
2,2,Dunedin,NZ,-45.8742,170.5036,68.52,66,60,8.23,broken clouds
3,3,Bathsheba,BB,13.2167,-59.5167,78.80,78,75,17.27,broken clouds
4,4,Albany,US,42.6001,-73.9662,36.00,73,100,0.85,overcast clouds
...,...,...,...,...,...,...,...,...,...,...
703,703,Bitung,ID,1.4451,125.1824,78.80,94,75,2.30,broken clouds
704,704,Lázaro Cárdenas,MX,17.9583,-102.2000,76.55,73,100,4.54,overcast clouds
705,705,Iraray,PH,8.9936,118.0511,81.68,76,15,3.04,few clouds
706,706,Yumen,CN,40.2833,97.2000,17.56,88,86,12.06,overcast clouds


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 30
What is the maximum temperature you would like for your trip? 60


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] > min_temp) & (city_data_df['Max Temp'] < max_temp)]

In [12]:
# 4a. Determine if there are any empty rows.
clean_df.isna().any()

Unnamed: 0             False
City                   False
Country                False
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Current Description    False
dtype: bool

In [14]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hasaki,JP,41.00,broken clouds,35.7333,140.8333,
4,Albany,US,36.00,overcast clouds,42.6001,-73.9662,
5,Lebu,CL,55.11,clear sky,-37.6167,-73.6500,
11,Ancud,CL,51.80,light rain,-41.8697,-73.8203,
12,Jamestown,US,34.00,clear sky,42.0970,-79.2353,
13,Punta Arenas,CL,55.40,few clouds,-53.1500,-70.9167,
20,Bethel,US,36.00,overcast clouds,41.3712,-73.4140,
33,Port Hardy,CA,46.40,light rain,50.6996,-127.4199,
37,Aljezur,PT,51.73,overcast clouds,37.3191,-8.8033,
43,Hagerstown,US,42.80,clear sky,39.6418,-77.7200,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [19]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
# hotels = requests.get(base_url, params=params).json()

# hotels

In [22]:

# 6b. Iterate through the DataFrame.
for index, row in tqdm(hotel_df.iterrows(),  total=hotel_df.shape[0]):
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = str(lat) + ', ' + str(lng)
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()

    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]        
    except Exception as e:
        print(e)
        'Hotel not found, skipping...'

Inubosaki Kanko Hotel



IndexError: list index out of range

In [49]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']

In [50]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [3]:
output_data_file = 'WeatherPy_vacation.csv'
clean_hotel_df = pd.read_csv(output_data_file, index_col=0)

In [4]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [5]:
# 11a. Add a marker layer for each city to the map.
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))